#4on4 : 2024 Power Ranking
---------------------------

## Dependencies

In [290]:
import json
import pandas as pd
import plotly.express as px

from sklearn.preprocessing import MinMaxScaler

## Functions

In [291]:
def load_data(file_path: str, category: str = None) -> pd.DataFrame:
    """Load data as pandas Dataframe, with basic information and selected statistics if needed
    :param file_path: path to data as JSON file to parse
    :param category: category of stats to select
    :return: pandas Dataframe with all information requested.
    """
    with open(file_path, 'r', encoding='utf-8') as j_file:
        json_data = json.load(j_file)

    with open('../data/stats_category.json', 'r', encoding='utf-8') as c_file:
        stats_categories = json.load(c_file)

    general_data_keys = ('artist_list', 'genre', 'label', 'title')
    renaming = {'s_id': 'songstats_id', 's_title': 'songstats_title'}

    ids = pd.DataFrame([{key: value for key, value in field['songstats_identifiers'].items()} for field in
                        json_data]).rename(columns=renaming)

    general_data = pd.DataFrame(
        [{key: value for key, value in field.items() if key in general_data_keys} for field in json_data])

    if not category:
        return pd.concat((ids, general_data), axis=1)

    stats_selected = pd.DataFrame(
        [{key: value for key, value in field['data'].items() if key in stats_categories[category]}
         for field in json_data]).fillna(0)

    return pd.concat((ids, general_data, stats_selected), axis=1)

In [292]:
def rescale_stats(df: pd.DataFrame, other_fields: list) -> pd.DataFrame:
    """Rescale stats for better comparaison
    :param df: input pandas Dataframe
    :param other_fields: field to not rescale
    :return: data rescaled.
    """
    df[df.columns.difference(other_fields)] = MinMaxScaler(feature_range=(0, 100)).fit_transform(
        df[df.columns.difference(other_fields)])

    return df


In [293]:
def compute_power(df: pd.DataFrame, other_fields: list) -> (pd.DataFrame, float):
    """Compute
    :param df: input pandas Dataframe
    :param other_fields: field to not rescale
    :return: data with power ranking computed and category weight
    """
    # "prw" stand for "Power Ranking Weight" : proportion of data available for each stat
    prw = df[df.columns.difference(other_fields)].astype(bool).sum(axis=0) / df.shape[0]
    pr = df[df.columns.difference(other_fields)].dot(prw) / prw.sum()

    category_weight = prw.sum() / prw.shape[0]

    pr_results = pd.concat((df, pr), axis=1)
    pr_results.rename({0: "PWR"}, axis=1, inplace=True)

    return pr_results, category_weight

In [294]:
def plot_bar(dataframe: pd.DataFrame, x: str, y: str, title: str, x_label: str, y_label: str, col_range: tuple):
    """Compute a Bar Chart using Plotly express
    :param dataframe: dataframe as input (has to be sorted)
    :param x: x value for bar chart (labels)
    :param y: y value for bar chart (feature)
    :param title: title of the chart
    :param x_label: x-axis label
    :param y_label: y-axis label
    :param col_range: color range for the chart
    :return: bar chart as plotly "Figure" object.
    """
    if x_label is None:
        x_label = x

    if y_label is None:
        y_label = y

    fig = px.bar(data_frame=dataframe, y=y, x=x,
                 labels={x: x_label, y: y_label},
                 title=title, color=y,
                 width=960, height=540, color_continuous_scale='orrd', range_color=col_range)

    fig.update_layout(xaxis={'categoryorder': 'total descending'})
    fig.update_coloraxes(showscale=False)
    return fig

## Data Loading & Pre-treatments

In [295]:
data = load_data(file_path='../data/data_2024.json')
general_field = data.columns.tolist()
data

songstats_id                songstats_title  \
0       qmr6e0bx                             16   
1       cxsruom1  17 (Ekko & Sidetrack Bootleg)   
2       847p1x5v             Never Let You Down   
3       4owlyi76                           4 AM   
4       a3ok10ce                             4D   
..           ...                            ...   
302     cqdneir6                Can You Feel It   
303     nwqit015              Curse of the Moai   
304     em4stqoh                       Himalaya   
305     wx93mcy6                     Next To Me   
306     0wpg1fj9             Hopelessly Falling   

                                           artist_list  \
0                      [blasterjaxx, hardwell, maddix]   
1                               [ekko & sidetrack, mk]   
2    [cimone, deadline, duskee, emily makis, fricti...   
3                                       [lexed, oshan]   
4                            [bad computer, ryan coss]   
..                                                 ...   
302                                      [retrovision]   
303                                      [exyl, nasko]   
304                                  [matisse & sadko]   
305                      [don diablo, gaby gerlis, mk]   
306                   [enrem, jonah sky, rico 56, ubo]   

                              genre                 label  \
0                     [hard techno]            [revealed]   
1                     [drum & bass]         [[no labels]]   
2                     [drum & bass]        [shogun audio]   
3    [future house / future bounce]  [future house cloud]   
4                   [electro house]          [monstercat]   
..                              ...                   ...   
302                    [bass house]               [stmpd]   
303        [drumstep / moombahcore]         [[no labels]]   
304             [progressive house]               [stmpd]   
305  [future house / future bounce]             [hexagon]   
306                 [melodic house]     [monstercat silk]   

                           title  
0                             16  
1    17 ekko & sidetrack bootleg  
2             never let you down  
3                           4 am  
4                             4d  
..                           ...  
302              can you feel it  
303            curse of the moai  
304                     himalaya  
305                   next to me  
306           hopelessly falling  

[307 rows x 6 columns]

In [296]:
# Stats by artist
data_art = data.explode('artist_list').reset_index(drop=True)
data_art.rename({'artist_list': 'artist'}, axis=1, inplace=True)

In [297]:
# Stats by labels
data_lab = data.explode('label').reset_index(drop=True)
data_lab = data_lab.loc[data_lab.label != '[no labels]']

In [298]:
# Stats by genre
data_gen = data.explode('genre').reset_index(drop=True)

## Basic statistics
### Artists of the year by number of release


In [299]:
data_art_count = (data_art[['artist', 'title']]
                  .groupby('artist', as_index=False)
                  .count()
                  .rename({'title': 'n_release'}, axis=1)
                  .sort_values('n_release', ascending=False)
                  .reset_index(drop=True))

data_art_count.describe()

n_release
count  433.000000
mean     1.348730
std      0.969848
min      1.000000
25%      1.000000
50%      1.000000
75%      1.000000
max      9.000000

In [300]:
bar_artists_release = plot_bar(dataframe=data_art_count.head(10),
                               x='artist',
                               y='n_release',
                               title='Number of release by Artist (Top 10)<br><sup>Remixes included</sup>',
                               x_label='Artists',
                               y_label='Number of releases',
                               col_range=(data_art_count.n_release.min(), data_art_count.n_release.max()))

bar_artists_release.show()


### Labels of the year by number of release

In [301]:
data_lab_count = (data_lab[['label', 'title']]
                  .groupby('label', as_index=False)
                  .count()
                  .rename({'title': 'n_release'}, axis=1)
                  .sort_values('n_release', ascending=False)
                  .reset_index(drop=True))

data_lab_count.describe()

n_release
count  89.000000
mean    3.303371
std     6.951789
min     1.000000
25%     1.000000
50%     1.000000
75%     2.000000
max    54.000000

In [302]:
bar_label_release = plot_bar(dataframe=data_lab_count.head(10),
                             x='label',
                             y='n_release',
                             title='Number of release by Label (Top 10)',
                             x_label='Labels',
                             y_label='Number of releases',
                             col_range=(data_lab_count.n_release.min(), data_lab_count.n_release.max()))
bar_label_release.show()

### Genre of the year by number of release


In [303]:
data_gen_count = (data_gen[['genre', 'title']]
                  .groupby('genre', as_index=False)
                  .count()
                  .rename({'title': 'n_release'}, axis=1)
                  .sort_values('n_release', ascending=False)
                  .reset_index(drop=True))

data_gen_count.describe()

n_release
count   39.000000
mean     8.128205
std     18.450759
min      1.000000
25%      1.500000
50%      3.000000
75%      5.500000
max    112.000000

In [304]:
bar_genre_release = plot_bar(dataframe=data_gen_count,
                             x='genre',
                             y='n_release',
                             title='Number of release by Genre',
                             x_label='Music Genres',
                             y_label='Number of releases',
                             col_range=(data_gen_count.n_release.min(), data_gen_count.n_release.max()))
bar_genre_release.show()

## Power Rankings by stats categories
### Loading data

In [305]:
data_cha = load_data(file_path='../data/data_2024.json', category='charts')
data_eng = load_data(file_path='../data/data_2024.json', category='engagement')
data_pla = load_data(file_path='../data/data_2024.json', category='playlists')
data_pop = load_data(file_path='../data/data_2024.json', category='popularity')
data_pro = load_data(file_path='../data/data_2024.json', category='professional_support')
data_rea = load_data(file_path='../data/data_2024.json', category='reach')
data_sho = load_data(file_path='../data/data_2024.json', category='shorts')
data_str = load_data(file_path='../data/data_2024.json', category='streams')

### Charts
#### Rescale statistics

In [306]:
data_cha = rescale_stats(data_cha, general_field)
data_cha

songstats_id                songstats_title  \
0       qmr6e0bx                             16   
1       cxsruom1  17 (Ekko & Sidetrack Bootleg)   
2       847p1x5v             Never Let You Down   
3       4owlyi76                           4 AM   
4       a3ok10ce                             4D   
..           ...                            ...   
302     cqdneir6                Can You Feel It   
303     nwqit015              Curse of the Moai   
304     em4stqoh                       Himalaya   
305     wx93mcy6                     Next To Me   
306     0wpg1fj9             Hopelessly Falling   

                                           artist_list  \
0                      [blasterjaxx, hardwell, maddix]   
1                               [ekko & sidetrack, mk]   
2    [cimone, deadline, duskee, emily makis, fricti...   
3                                       [lexed, oshan]   
4                            [bad computer, ryan coss]   
..                                                 ...   
302                                      [retrovision]   
303                                      [exyl, nasko]   
304                                  [matisse & sadko]   
305                      [don diablo, gaby gerlis, mk]   
306                   [enrem, jonah sky, rico 56, ubo]   

                              genre                 label  \
0                     [hard techno]            [revealed]   
1                     [drum & bass]         [[no labels]]   
2                     [drum & bass]        [shogun audio]   
3    [future house / future bounce]  [future house cloud]   
4                   [electro house]          [monstercat]   
..                              ...                   ...   
302                    [bass house]               [stmpd]   
303        [drumstep / moombahcore]         [[no labels]]   
304             [progressive house]               [stmpd]   
305  [future house / future bounce]             [hexagon]   
306                 [melodic house]     [monstercat silk]   

                           title  amazon_charts_total  \
0                             16                  0.0   
1    17 ekko & sidetrack bootleg                  0.0   
2             never let you down                  0.0   
3                           4 am                  0.0   
4                             4d                  0.0   
..                           ...                  ...   
302              can you feel it                  0.0   
303            curse of the moai                  0.0   
304                     himalaya                  0.0   
305                   next to me                  0.0   
306           hopelessly falling                  0.0   

     apple_music_charts_total  soundcloud_charts_total  spotify_charts_total  \
0                    9.433962                      0.0                   0.0   
1                    0.000000                      0.0                   0.0   
2                    0.000000                      0.0                   0.0   
3                    0.628931                      0.0                   0.0   
4                    1.257862                      0.0                   0.0   
..                        ...                      ...                   ...   
302                  0.000000                      0.0                   0.0   
303                  0.000000                      0.0                   0.0   
304                  0.000000                      0.0                   0.0   
305                  2.515723                      0.0                   0.0   
306                  0.000000                      0.0                   0.0   

     tidal_charts_total  tiktok_charts_total  traxsource_dj_charts_total  \
0                   0.0                  0.0                         0.0   
1                   0.0                  0.0                         0.0   
2                   0.0                  0.0                         0.0   
3          

#### Weighting by recorded by stats

In [307]:
pr_cha, w_cha = compute_power(data_cha, general_field)  # 'w_cha' is calculated for the final step of the Power Ranking
pr_cha

songstats_id                songstats_title  \
0       qmr6e0bx                             16   
1       cxsruom1  17 (Ekko & Sidetrack Bootleg)   
2       847p1x5v             Never Let You Down   
3       4owlyi76                           4 AM   
4       a3ok10ce                             4D   
..           ...                            ...   
302     cqdneir6                Can You Feel It   
303     nwqit015              Curse of the Moai   
304     em4stqoh                       Himalaya   
305     wx93mcy6                     Next To Me   
306     0wpg1fj9             Hopelessly Falling   

                                           artist_list  \
0                      [blasterjaxx, hardwell, maddix]   
1                               [ekko & sidetrack, mk]   
2    [cimone, deadline, duskee, emily makis, fricti...   
3                                       [lexed, oshan]   
4                            [bad computer, ryan coss]   
..                                                 ...   
302                                      [retrovision]   
303                                      [exyl, nasko]   
304                                  [matisse & sadko]   
305                      [don diablo, gaby gerlis, mk]   
306                   [enrem, jonah sky, rico 56, ubo]   

                              genre                 label  \
0                     [hard techno]            [revealed]   
1                     [drum & bass]         [[no labels]]   
2                     [drum & bass]        [shogun audio]   
3    [future house / future bounce]  [future house cloud]   
4                   [electro house]          [monstercat]   
..                              ...                   ...   
302                    [bass house]               [stmpd]   
303        [drumstep / moombahcore]         [[no labels]]   
304             [progressive house]               [stmpd]   
305  [future house / future bounce]             [hexagon]   
306                 [melodic house]     [monstercat silk]   

                           title  amazon_charts_total  \
0                             16                  0.0   
1    17 ekko & sidetrack bootleg                  0.0   
2             never let you down                  0.0   
3                           4 am                  0.0   
4                             4d                  0.0   
..                           ...                  ...   
302              can you feel it                  0.0   
303            curse of the moai                  0.0   
304                     himalaya                  0.0   
305                   next to me                  0.0   
306           hopelessly falling                  0.0   

     apple_music_charts_total  soundcloud_charts_total  spotify_charts_total  \
0                    9.433962                      0.0                   0.0   
1                    0.000000                      0.0                   0.0   
2                    0.000000                      0.0                   0.0   
3                    0.628931                      0.0                   0.0   
4                    1.257862                      0.0                   0.0   
..                        ...                      ...                   ...   
302                  0.000000                      0.0                   0.0   
303                  0.000000                      0.0                   0.0   
304                  0.000000                      0.0                   0.0   
305                  2.515723                      0.0                   0.0   
306                  0.000000                      0.0                   0.0   

     tidal_charts_total  tiktok_charts_total  traxsource_dj_charts_total  \
0                   0.0                  0.0                         0.0   
1                   0.0                  0.0                         0.0   
2                   0.0                  0.0                         0.0   
3          

#### Graphical Representation

In [308]:
bar_pr_cha = plot_bar(dataframe=pr_cha.sort_values('PWR', ascending=False).head(20),
                      x='songstats_title',
                      y='PWR',
                      title='Charts Power Ranking (Top 20)',
                      x_label='Track',
                      y_label='Power',
                      col_range=(pr_cha.PWR.min(), pr_cha.PWR.max()))
bar_pr_cha.show()

### Engagement
#### Rescale statistics

In [309]:
data_eng = rescale_stats(data_eng, general_field)
data_eng

songstats_id                songstats_title  \
0       qmr6e0bx                             16   
1       cxsruom1  17 (Ekko & Sidetrack Bootleg)   
2       847p1x5v             Never Let You Down   
3       4owlyi76                           4 AM   
4       a3ok10ce                             4D   
..           ...                            ...   
302     cqdneir6                Can You Feel It   
303     nwqit015              Curse of the Moai   
304     em4stqoh                       Himalaya   
305     wx93mcy6                     Next To Me   
306     0wpg1fj9             Hopelessly Falling   

                                           artist_list  \
0                      [blasterjaxx, hardwell, maddix]   
1                               [ekko & sidetrack, mk]   
2    [cimone, deadline, duskee, emily makis, fricti...   
3                                       [lexed, oshan]   
4                            [bad computer, ryan coss]   
..                                                 ...   
302                                      [retrovision]   
303                                      [exyl, nasko]   
304                                  [matisse & sadko]   
305                      [don diablo, gaby gerlis, mk]   
306                   [enrem, jonah sky, rico 56, ubo]   

                              genre                 label  \
0                     [hard techno]            [revealed]   
1                     [drum & bass]         [[no labels]]   
2                     [drum & bass]        [shogun audio]   
3    [future house / future bounce]  [future house cloud]   
4                   [electro house]          [monstercat]   
..                              ...                   ...   
302                    [bass house]               [stmpd]   
303        [drumstep / moombahcore]         [[no labels]]   
304             [progressive house]               [stmpd]   
305  [future house / future bounce]             [hexagon]   
306                 [melodic house]     [monstercat silk]   

                           title  soundcloud_engagement_rate_total  \
0                             16                         27.522936   
1    17 ekko & sidetrack bootleg                         35.779817   
2             never let you down                         51.376147   
3                           4 am                         29.357798   
4                             4d                         33.027523   
..                           ...                               ...   
302              can you feel it                          0.000000   
303            curse of the moai                          0.000000   
304                     himalaya                        100.000000   
305                   next to me                          0.000000   
306           hopelessly falling                         33.027523   

     tiktok_engagement_rate_total  youtube_engagement_rate_total  
0                       22.983871                      25.641026  
1                        0.000000                      18.803419  
2                       48.387097                      27.350427  
3                        0.000000                      17.948718  
4                       18.548387                      15.384615  
..                            ...                            ...  
302                      0.000000                      88.888889  
303                      0.000000                       0.000000  
304                     39.919355                      70.940171  
305                     49.596774                      35.042735  
306                      0.000000                     100.000000  

[307 rows x 9 columns]

#### Weighting by recorded by stats

In [310]:
pr_eng, w_eng = compute_power(data_eng, general_field)  # 'w_eng' is calculated for the final step of the Power Ranking
pr_eng

songstats_id                songstats_title  \
0       qmr6e0bx                             16   
1       cxsruom1  17 (Ekko & Sidetrack Bootleg)   
2       847p1x5v             Never Let You Down   
3       4owlyi76                           4 AM   
4       a3ok10ce                             4D   
..           ...                            ...   
302     cqdneir6                Can You Feel It   
303     nwqit015              Curse of the Moai   
304     em4stqoh                       Himalaya   
305     wx93mcy6                     Next To Me   
306     0wpg1fj9             Hopelessly Falling   

                                           artist_list  \
0                      [blasterjaxx, hardwell, maddix]   
1                               [ekko & sidetrack, mk]   
2    [cimone, deadline, duskee, emily makis, fricti...   
3                                       [lexed, oshan]   
4                            [bad computer, ryan coss]   
..                                                 ...   
302                                      [retrovision]   
303                                      [exyl, nasko]   
304                                  [matisse & sadko]   
305                      [don diablo, gaby gerlis, mk]   
306                   [enrem, jonah sky, rico 56, ubo]   

                              genre                 label  \
0                     [hard techno]            [revealed]   
1                     [drum & bass]         [[no labels]]   
2                     [drum & bass]        [shogun audio]   
3    [future house / future bounce]  [future house cloud]   
4                   [electro house]          [monstercat]   
..                              ...                   ...   
302                    [bass house]               [stmpd]   
303        [drumstep / moombahcore]         [[no labels]]   
304             [progressive house]               [stmpd]   
305  [future house / future bounce]             [hexagon]   
306                 [melodic house]     [monstercat silk]   

                           title  soundcloud_engagement_rate_total  \
0                             16                         27.522936   
1    17 ekko & sidetrack bootleg                         35.779817   
2             never let you down                         51.376147   
3                           4 am                         29.357798   
4                             4d                         33.027523   
..                           ...                               ...   
302              can you feel it                          0.000000   
303            curse of the moai                          0.000000   
304                     himalaya                        100.000000   
305                   next to me                          0.000000   
306           hopelessly falling                         33.027523   

     tiktok_engagement_rate_total  youtube_engagement_rate_total        PWR  
0                       22.983871                      25.641026  25.544699  
1                        0.000000                      18.803419  19.380532  
2                       48.387097                      27.350427  41.503941  
3                        0.000000                      17.948718  16.849796  
4                       18.548387                      15.384615  22.339668  
..                            ...                            ...        ...  
302                      0.000000                      88.888889  33.429237  
303                      0.000000                       0.000000   0.000000  
304                     39.919355                      70.940171  72.254557  
305                     49.596774                      35.042735  27.061038  
306                      0.000000                     100.000000  48.970011  

[307 rows x 10 columns]

#### Graphical Representation

In [311]:
bar_pr_eng = plot_bar(dataframe=pr_eng.sort_values('PWR', ascending=False).head(20),
                      x='songstats_title',
                      y='PWR',
                      title='Engagement Power Ranking (Top 20)',
                      x_label='Track',
                      y_label='Power',
                      col_range=(pr_eng.PWR.min(), pr_eng.PWR.max()))
bar_pr_eng.show()

### Playlists
#### Rescale statistics

In [315]:
data_pla = rescale_stats(data_pla, general_field)
data_pla

songstats_id                songstats_title  \
0       qmr6e0bx                             16   
1       cxsruom1  17 (Ekko & Sidetrack Bootleg)   
2       847p1x5v             Never Let You Down   
3       4owlyi76                           4 AM   
4       a3ok10ce                             4D   
..           ...                            ...   
302     cqdneir6                Can You Feel It   
303     nwqit015              Curse of the Moai   
304     em4stqoh                       Himalaya   
305     wx93mcy6                     Next To Me   
306     0wpg1fj9             Hopelessly Falling   

                                           artist_list  \
0                      [blasterjaxx, hardwell, maddix]   
1                               [ekko & sidetrack, mk]   
2    [cimone, deadline, duskee, emily makis, fricti...   
3                                       [lexed, oshan]   
4                            [bad computer, ryan coss]   
..                                                 ...   
302                                      [retrovision]   
303                                      [exyl, nasko]   
304                                  [matisse & sadko]   
305                      [don diablo, gaby gerlis, mk]   
306                   [enrem, jonah sky, rico 56, ubo]   

                              genre                 label  \
0                     [hard techno]            [revealed]   
1                     [drum & bass]         [[no labels]]   
2                     [drum & bass]        [shogun audio]   
3    [future house / future bounce]  [future house cloud]   
4                   [electro house]          [monstercat]   
..                              ...                   ...   
302                    [bass house]               [stmpd]   
303        [drumstep / moombahcore]         [[no labels]]   
304             [progressive house]               [stmpd]   
305  [future house / future bounce]             [hexagon]   
306                 [melodic house]     [monstercat silk]   

                           title  amazon_playlists_editorial_total  \
0                             16                          9.665428   
1    17 ekko & sidetrack bootleg                          0.000000   
2             never let you down                          0.000000   
3                           4 am                          0.000000   
4                             4d                          3.345725   
..                           ...                               ...   
302              can you feel it                          0.000000   
303            curse of the moai                          0.000000   
304                     himalaya                          0.000000   
305                   next to me                          7.806691   
306           hopelessly falling                          0.000000   

     apple_music_playlists_editorial_total  deezer_playlists_editorial_total  \
0                                17.142857                         19.230769   
1                                 0.000000                          0.000000   
2                                 0.000000                          0.000000   
3                                 0.000000                          0.000000   
4                                 8.571429                          0.000000   
..                                     ...                               ...   
302                               0.000000                          0.000000   
303                               0.000000                          0.000000   
304                               0.000000                          0.000000   
305                              11.428571                         26.923077   
306                               0.000000                          0.000000   

     spotify_playlists_editorial_total  tidal_playlists_editorial_total  \
0                             5.940594                              0.0   
1        

#### Weighting by recorded by stats

In [320]:
pr_pla, w_pla = compute_power(data_pla, general_field)  # 'w_pla' is calculated for the final step of the Power Ranking
pr_pla

songstats_id                songstats_title  \
0       qmr6e0bx                             16   
1       cxsruom1  17 (Ekko & Sidetrack Bootleg)   
2       847p1x5v             Never Let You Down   
3       4owlyi76                           4 AM   
4       a3ok10ce                             4D   
..           ...                            ...   
302     cqdneir6                Can You Feel It   
303     nwqit015              Curse of the Moai   
304     em4stqoh                       Himalaya   
305     wx93mcy6                     Next To Me   
306     0wpg1fj9             Hopelessly Falling   

                                           artist_list  \
0                      [blasterjaxx, hardwell, maddix]   
1                               [ekko & sidetrack, mk]   
2    [cimone, deadline, duskee, emily makis, fricti...   
3                                       [lexed, oshan]   
4                            [bad computer, ryan coss]   
..                                                 ...   
302                                      [retrovision]   
303                                      [exyl, nasko]   
304                                  [matisse & sadko]   
305                      [don diablo, gaby gerlis, mk]   
306                   [enrem, jonah sky, rico 56, ubo]   

                              genre                 label  \
0                     [hard techno]            [revealed]   
1                     [drum & bass]         [[no labels]]   
2                     [drum & bass]        [shogun audio]   
3    [future house / future bounce]  [future house cloud]   
4                   [electro house]          [monstercat]   
..                              ...                   ...   
302                    [bass house]               [stmpd]   
303        [drumstep / moombahcore]         [[no labels]]   
304             [progressive house]               [stmpd]   
305  [future house / future bounce]             [hexagon]   
306                 [melodic house]     [monstercat silk]   

                           title  amazon_playlists_editorial_total  \
0                             16                          9.665428   
1    17 ekko & sidetrack bootleg                          0.000000   
2             never let you down                          0.000000   
3                           4 am                          0.000000   
4                             4d                          3.345725   
..                           ...                               ...   
302              can you feel it                          0.000000   
303            curse of the moai                          0.000000   
304                     himalaya                          0.000000   
305                   next to me                          7.806691   
306           hopelessly falling                          0.000000   

     apple_music_playlists_editorial_total  deezer_playlists_editorial_total  \
0                                17.142857                         19.230769   
1                                 0.000000                          0.000000   
2                                 0.000000                          0.000000   
3                                 0.000000                          0.000000   
4                                 8.571429                          0.000000   
..                                     ...                               ...   
302                               0.000000                          0.000000   
303                               0.000000                          0.000000   
304                               0.000000                          0.000000   
305                              11.428571                         26.923077   
306                               0.000000                          0.000000   

     spotify_playlists_editorial_total  tidal_playlists_editorial_total  \
0                             5.940594                              0.0   
1        

#### Graphical Representation

In [327]:
bar_pr_pla = plot_bar(dataframe=pr_pla.sort_values('PWR', ascending=False).head(20),
                      x='songstats_title',
                      y='PWR',
                      title='Playlist Power Ranking (Top 20)',
                      x_label='Track',
                      y_label='Power',
                      col_range=(pr_pla.PWR.min(), pr_pla.PWR.max()))
bar_pr_pla.show()

### Popularity
#### Rescale statistics

In [312]:
data_pop = rescale_stats(data_pop, general_field)
data_pop

songstats_id                songstats_title  \
0       qmr6e0bx                             16   
1       cxsruom1  17 (Ekko & Sidetrack Bootleg)   
2       847p1x5v             Never Let You Down   
3       4owlyi76                           4 AM   
4       a3ok10ce                             4D   
..           ...                            ...   
302     cqdneir6                Can You Feel It   
303     nwqit015              Curse of the Moai   
304     em4stqoh                       Himalaya   
305     wx93mcy6                     Next To Me   
306     0wpg1fj9             Hopelessly Falling   

                                           artist_list  \
0                      [blasterjaxx, hardwell, maddix]   
1                               [ekko & sidetrack, mk]   
2    [cimone, deadline, duskee, emily makis, fricti...   
3                                       [lexed, oshan]   
4                            [bad computer, ryan coss]   
..                                                 ...   
302                                      [retrovision]   
303                                      [exyl, nasko]   
304                                  [matisse & sadko]   
305                      [don diablo, gaby gerlis, mk]   
306                   [enrem, jonah sky, rico 56, ubo]   

                              genre                 label  \
0                     [hard techno]            [revealed]   
1                     [drum & bass]         [[no labels]]   
2                     [drum & bass]        [shogun audio]   
3    [future house / future bounce]  [future house cloud]   
4                   [electro house]          [monstercat]   
..                              ...                   ...   
302                    [bass house]               [stmpd]   
303        [drumstep / moombahcore]         [[no labels]]   
304             [progressive house]               [stmpd]   
305  [future house / future bounce]             [hexagon]   
306                 [melodic house]     [monstercat silk]   

                           title  deezer_popularity_peak  \
0                             16               80.808081   
1    17 ekko & sidetrack bootleg                0.000000   
2             never let you down               21.212121   
3                           4 am               13.131313   
4                             4d               34.343434   
..                           ...                     ...   
302              can you feel it               10.101010   
303            curse of the moai                0.000000   
304                     himalaya               10.101010   
305                   next to me               13.131313   
306           hopelessly falling               21.212121   

     spotify_popularity_peak  tidal_popularity_peak  
0                  78.481013                50.0000  
1                   0.000000                 0.0000  
2                  43.037975                 0.0000  
3                  25.316456                 0.0000  
4                  59.493671                10.9375  
..                       ...                    ...  
302                40.506329                 0.0000  
303                44.303797                 0.0000  
304                49.367089                 4.6875  
305                68.354430                10.9375  
306                26.582278                 0.0000  

[307 rows x 9 columns]

#### Weighting by recorded by stats

In [313]:
pr_pop, w_pop = compute_power(data_pop, general_field)  # 'w_pop' is calculated for the final step of the Power Ranking
pr_pop

songstats_id                songstats_title  \
0       qmr6e0bx                             16   
1       cxsruom1  17 (Ekko & Sidetrack Bootleg)   
2       847p1x5v             Never Let You Down   
3       4owlyi76                           4 AM   
4       a3ok10ce                             4D   
..           ...                            ...   
302     cqdneir6                Can You Feel It   
303     nwqit015              Curse of the Moai   
304     em4stqoh                       Himalaya   
305     wx93mcy6                     Next To Me   
306     0wpg1fj9             Hopelessly Falling   

                                           artist_list  \
0                      [blasterjaxx, hardwell, maddix]   
1                               [ekko & sidetrack, mk]   
2    [cimone, deadline, duskee, emily makis, fricti...   
3                                       [lexed, oshan]   
4                            [bad computer, ryan coss]   
..                                                 ...   
302                                      [retrovision]   
303                                      [exyl, nasko]   
304                                  [matisse & sadko]   
305                      [don diablo, gaby gerlis, mk]   
306                   [enrem, jonah sky, rico 56, ubo]   

                              genre                 label  \
0                     [hard techno]            [revealed]   
1                     [drum & bass]         [[no labels]]   
2                     [drum & bass]        [shogun audio]   
3    [future house / future bounce]  [future house cloud]   
4                   [electro house]          [monstercat]   
..                              ...                   ...   
302                    [bass house]               [stmpd]   
303        [drumstep / moombahcore]         [[no labels]]   
304             [progressive house]               [stmpd]   
305  [future house / future bounce]             [hexagon]   
306                 [melodic house]     [monstercat silk]   

                           title  deezer_popularity_peak  \
0                             16               80.808081   
1    17 ekko & sidetrack bootleg                0.000000   
2             never let you down               21.212121   
3                           4 am               13.131313   
4                             4d               34.343434   
..                           ...                     ...   
302              can you feel it               10.101010   
303            curse of the moai                0.000000   
304                     himalaya               10.101010   
305                   next to me               13.131313   
306           hopelessly falling               21.212121   

     spotify_popularity_peak  tidal_popularity_peak        PWR  
0                  78.481013                50.0000  72.421201  
1                   0.000000                 0.0000   0.000000  
2                  43.037975                 0.0000  24.439029  
3                  25.316456                 0.0000  14.619412  
4                  59.493671                10.9375  38.314074  
..                       ...                    ...        ...  
302                40.506329                 0.0000  19.326611  
303                44.303797                 0.0000  17.022287  
304                49.367089                 4.6875  23.871108  
305                68.354430                10.9375  33.815439  
306                26.582278                 0.0000  18.116465  

[307 rows x 10 columns]

#### Graphical Representation

In [314]:
bar_pr_pop = plot_bar(dataframe=pr_pop.sort_values('PWR', ascending=False).head(20),
                      x='songstats_title',
                      y='PWR',
                      title='Popularity Power Ranking (Top 20)',
                      x_label='Track',
                      y_label='Power',
                      col_range=(pr_pop.PWR.min(), pr_pop.PWR.max()))
bar_pr_pop.show()

### Professional Support
#### Rescale statistics

In [316]:
data_pro = rescale_stats(data_pro, general_field)
data_pro

songstats_id                songstats_title  \
0       qmr6e0bx                             16   
1       cxsruom1  17 (Ekko & Sidetrack Bootleg)   
2       847p1x5v             Never Let You Down   
3       4owlyi76                           4 AM   
4       a3ok10ce                             4D   
..           ...                            ...   
302     cqdneir6                Can You Feel It   
303     nwqit015              Curse of the Moai   
304     em4stqoh                       Himalaya   
305     wx93mcy6                     Next To Me   
306     0wpg1fj9             Hopelessly Falling   

                                           artist_list  \
0                      [blasterjaxx, hardwell, maddix]   
1                               [ekko & sidetrack, mk]   
2    [cimone, deadline, duskee, emily makis, fricti...   
3                                       [lexed, oshan]   
4                            [bad computer, ryan coss]   
..                                                 ...   
302                                      [retrovision]   
303                                      [exyl, nasko]   
304                                  [matisse & sadko]   
305                      [don diablo, gaby gerlis, mk]   
306                   [enrem, jonah sky, rico 56, ubo]   

                              genre                 label  \
0                     [hard techno]            [revealed]   
1                     [drum & bass]         [[no labels]]   
2                     [drum & bass]        [shogun audio]   
3    [future house / future bounce]  [future house cloud]   
4                   [electro house]          [monstercat]   
..                              ...                   ...   
302                    [bass house]               [stmpd]   
303        [drumstep / moombahcore]         [[no labels]]   
304             [progressive house]               [stmpd]   
305  [future house / future bounce]             [hexagon]   
306                 [melodic house]     [monstercat silk]   

                           title  1001tracklists_unique_support  \
0                             16                      17.777778   
1    17 ekko & sidetrack bootleg                      13.333333   
2             never let you down                       0.000000   
3                           4 am                       0.000000   
4                             4d                       3.555556   
..                           ...                            ...   
302              can you feel it                       2.222222   
303            curse of the moai                       0.000000   
304                     himalaya                       3.111111   
305                   next to me                      11.555556   
306           hopelessly falling                       0.000000   

     beatport_dj_charts_total  
0                   50.000000  
1                    0.000000  
2                    0.000000  
3                    0.000000  
4                    0.000000  
..                        ...  
302                  1.428571  
303                  0.000000  
304                  2.857143  
305                  1.428571  
306                  0.000000  

[307 rows x 8 columns]

#### Weighting by recorded by stats

In [323]:
pr_pro, w_pro = compute_power(data_pro, general_field)  # 'w_pro' is calculated for the final step of the Power Ranking
pr_pro

songstats_id                songstats_title  \
0       qmr6e0bx                             16   
1       cxsruom1  17 (Ekko & Sidetrack Bootleg)   
2       847p1x5v             Never Let You Down   
3       4owlyi76                           4 AM   
4       a3ok10ce                             4D   
..           ...                            ...   
302     cqdneir6                Can You Feel It   
303     nwqit015              Curse of the Moai   
304     em4stqoh                       Himalaya   
305     wx93mcy6                     Next To Me   
306     0wpg1fj9             Hopelessly Falling   

                                           artist_list  \
0                      [blasterjaxx, hardwell, maddix]   
1                               [ekko & sidetrack, mk]   
2    [cimone, deadline, duskee, emily makis, fricti...   
3                                       [lexed, oshan]   
4                            [bad computer, ryan coss]   
..                                                 ...   
302                                      [retrovision]   
303                                      [exyl, nasko]   
304                                  [matisse & sadko]   
305                      [don diablo, gaby gerlis, mk]   
306                   [enrem, jonah sky, rico 56, ubo]   

                              genre                 label  \
0                     [hard techno]            [revealed]   
1                     [drum & bass]         [[no labels]]   
2                     [drum & bass]        [shogun audio]   
3    [future house / future bounce]  [future house cloud]   
4                   [electro house]          [monstercat]   
..                              ...                   ...   
302                    [bass house]               [stmpd]   
303        [drumstep / moombahcore]         [[no labels]]   
304             [progressive house]               [stmpd]   
305  [future house / future bounce]             [hexagon]   
306                 [melodic house]     [monstercat silk]   

                           title  1001tracklists_unique_support  \
0                             16                      17.777778   
1    17 ekko & sidetrack bootleg                      13.333333   
2             never let you down                       0.000000   
3                           4 am                       0.000000   
4                             4d                       3.555556   
..                           ...                            ...   
302              can you feel it                       2.222222   
303            curse of the moai                       0.000000   
304                     himalaya                       3.111111   
305                   next to me                      11.555556   
306           hopelessly falling                       0.000000   

     beatport_dj_charts_total        PWR  
0                   50.000000  31.780893  
1                    0.000000   7.538941  
2                    0.000000   0.000000  
3                    0.000000   0.000000  
4                    0.000000   2.010384  
..                        ...        ...  
302                  1.428571   1.877318  
303                  0.000000   0.000000  
304                  2.857143   3.000742  
305                  1.428571   7.154576  
306                  0.000000   0.000000  

[307 rows x 9 columns]

#### Graphical Representation

In [328]:
bar_pr_pro = plot_bar(dataframe=pr_pro.sort_values('PWR', ascending=False).head(20),
                      x='songstats_title',
                      y='PWR',
                      title='Pro. Support Power Ranking (Top 20)',
                      x_label='Track',
                      y_label='Power',
                      col_range=(pr_pro.PWR.min(), pr_pro.PWR.max()))
bar_pr_pro.show()

### Reach
#### Rescale statistics

In [317]:
data_rea = rescale_stats(data_rea, general_field)
data_rea

songstats_id                songstats_title  \
0       qmr6e0bx                             16   
1       cxsruom1  17 (Ekko & Sidetrack Bootleg)   
2       847p1x5v             Never Let You Down   
3       4owlyi76                           4 AM   
4       a3ok10ce                             4D   
..           ...                            ...   
302     cqdneir6                Can You Feel It   
303     nwqit015              Curse of the Moai   
304     em4stqoh                       Himalaya   
305     wx93mcy6                     Next To Me   
306     0wpg1fj9             Hopelessly Falling   

                                           artist_list  \
0                      [blasterjaxx, hardwell, maddix]   
1                               [ekko & sidetrack, mk]   
2    [cimone, deadline, duskee, emily makis, fricti...   
3                                       [lexed, oshan]   
4                            [bad computer, ryan coss]   
..                                                 ...   
302                                      [retrovision]   
303                                      [exyl, nasko]   
304                                  [matisse & sadko]   
305                      [don diablo, gaby gerlis, mk]   
306                   [enrem, jonah sky, rico 56, ubo]   

                              genre                 label  \
0                     [hard techno]            [revealed]   
1                     [drum & bass]         [[no labels]]   
2                     [drum & bass]        [shogun audio]   
3    [future house / future bounce]  [future house cloud]   
4                   [electro house]          [monstercat]   
..                              ...                   ...   
302                    [bass house]               [stmpd]   
303        [drumstep / moombahcore]         [[no labels]]   
304             [progressive house]               [stmpd]   
305  [future house / future bounce]             [hexagon]   
306                 [melodic house]     [monstercat silk]   

                           title  deezer_playlist_reach_total  \
0                             16                     5.997800   
1    17 ekko & sidetrack bootleg                     0.000000   
2             never let you down                     0.002665   
3                           4 am                     0.006059   
4                             4d                     0.003569   
..                           ...                          ...   
302              can you feel it                     0.003569   
303            curse of the moai                     0.000000   
304                     himalaya                     0.014284   
305                   next to me                    65.406125   
306           hopelessly falling                     0.000000   

     spotify_playlist_reach_total  
0                       10.117432  
1                        0.000000  
2                        0.702096  
3                        0.560306  
4                        0.773180  
..                            ...  
302                      2.158997  
303                      0.028646  
304                      1.280433  
305                     25.078644  
306                      0.108823  

[307 rows x 8 columns]

#### Weighting by recorded by stats

In [324]:
pr_rea, w_rea = compute_power(data_rea, general_field)  # 'w_rea' is calculated for the final step of the Power Ranking
pr_rea

songstats_id                songstats_title  \
0       qmr6e0bx                             16   
1       cxsruom1  17 (Ekko & Sidetrack Bootleg)   
2       847p1x5v             Never Let You Down   
3       4owlyi76                           4 AM   
4       a3ok10ce                             4D   
..           ...                            ...   
302     cqdneir6                Can You Feel It   
303     nwqit015              Curse of the Moai   
304     em4stqoh                       Himalaya   
305     wx93mcy6                     Next To Me   
306     0wpg1fj9             Hopelessly Falling   

                                           artist_list  \
0                      [blasterjaxx, hardwell, maddix]   
1                               [ekko & sidetrack, mk]   
2    [cimone, deadline, duskee, emily makis, fricti...   
3                                       [lexed, oshan]   
4                            [bad computer, ryan coss]   
..                                                 ...   
302                                      [retrovision]   
303                                      [exyl, nasko]   
304                                  [matisse & sadko]   
305                      [don diablo, gaby gerlis, mk]   
306                   [enrem, jonah sky, rico 56, ubo]   

                              genre                 label  \
0                     [hard techno]            [revealed]   
1                     [drum & bass]         [[no labels]]   
2                     [drum & bass]        [shogun audio]   
3    [future house / future bounce]  [future house cloud]   
4                   [electro house]          [monstercat]   
..                              ...                   ...   
302                    [bass house]               [stmpd]   
303        [drumstep / moombahcore]         [[no labels]]   
304             [progressive house]               [stmpd]   
305  [future house / future bounce]             [hexagon]   
306                 [melodic house]     [monstercat silk]   

                           title  deezer_playlist_reach_total  \
0                             16                     5.997800   
1    17 ekko & sidetrack bootleg                     0.000000   
2             never let you down                     0.002665   
3                           4 am                     0.006059   
4                             4d                     0.003569   
..                           ...                          ...   
302              can you feel it                     0.003569   
303            curse of the moai                     0.000000   
304                     himalaya                     0.014284   
305                   next to me                    65.406125   
306           hopelessly falling                     0.000000   

     spotify_playlist_reach_total        PWR  
0                       10.117432   8.464595  
1                        0.000000   0.000000  
2                        0.702096   0.421478  
3                        0.560306   0.337937  
4                        0.773180   0.464404  
..                            ...        ...  
302                      2.158997   1.294219  
303                      0.028646   0.017153  
304                      1.280433   0.772442  
305                     25.078644  41.258420  
306                      0.108823   0.065162  

[307 rows x 9 columns]

#### Graphical Representation

In [329]:
bar_pr_rea = plot_bar(dataframe=pr_rea.sort_values('PWR', ascending=False).head(20),
                      x='songstats_title',
                      y='PWR',
                      title='Reach Power Ranking (Top 20)',
                      x_label='Track',
                      y_label='Power',
                      col_range=(pr_rea.PWR.min(), pr_rea.PWR.max()))
bar_pr_rea.show()

### Shorts
#### Rescale statistics

In [318]:
data_sho = rescale_stats(data_sho, general_field)
data_sho

songstats_id                songstats_title  \
0       qmr6e0bx                             16   
1       cxsruom1  17 (Ekko & Sidetrack Bootleg)   
2       847p1x5v             Never Let You Down   
3       4owlyi76                           4 AM   
4       a3ok10ce                             4D   
..           ...                            ...   
302     cqdneir6                Can You Feel It   
303     nwqit015              Curse of the Moai   
304     em4stqoh                       Himalaya   
305     wx93mcy6                     Next To Me   
306     0wpg1fj9             Hopelessly Falling   

                                           artist_list  \
0                      [blasterjaxx, hardwell, maddix]   
1                               [ekko & sidetrack, mk]   
2    [cimone, deadline, duskee, emily makis, fricti...   
3                                       [lexed, oshan]   
4                            [bad computer, ryan coss]   
..                                                 ...   
302                                      [retrovision]   
303                                      [exyl, nasko]   
304                                  [matisse & sadko]   
305                      [don diablo, gaby gerlis, mk]   
306                   [enrem, jonah sky, rico 56, ubo]   

                              genre                 label  \
0                     [hard techno]            [revealed]   
1                     [drum & bass]         [[no labels]]   
2                     [drum & bass]        [shogun audio]   
3    [future house / future bounce]  [future house cloud]   
4                   [electro house]          [monstercat]   
..                              ...                   ...   
302                    [bass house]               [stmpd]   
303        [drumstep / moombahcore]         [[no labels]]   
304             [progressive house]               [stmpd]   
305  [future house / future bounce]             [hexagon]   
306                 [melodic house]     [monstercat silk]   

                           title  tiktok_views_total  \
0                             16            0.195087   
1    17 ekko & sidetrack bootleg            0.000000   
2             never let you down            0.003769   
3                           4 am            0.000000   
4                             4d            0.002297   
..                           ...                 ...   
302              can you feel it            0.000000   
303            curse of the moai            0.000000   
304                     himalaya            0.005572   
305                   next to me            0.001739   
306           hopelessly falling            0.000000   

     youtube_short_views_total  
0                     0.008558  
1                     0.000000  
2                     0.013441  
3                     0.000000  
4                     0.034079  
..                         ...  
302                   0.000000  
303                   0.000000  
304                   0.000130  
305                   0.000000  
306                   0.000000  

[307 rows x 8 columns]

#### Weighting by recorded by stats

In [332]:
pr_sho, w_sho = compute_power(data_sho, general_field)  # 'w_sho' is calculated for the final step of the Power Ranking
pr_sho

songstats_id                songstats_title  \
0       qmr6e0bx                             16   
1       cxsruom1  17 (Ekko & Sidetrack Bootleg)   
2       847p1x5v             Never Let You Down   
3       4owlyi76                           4 AM   
4       a3ok10ce                             4D   
..           ...                            ...   
302     cqdneir6                Can You Feel It   
303     nwqit015              Curse of the Moai   
304     em4stqoh                       Himalaya   
305     wx93mcy6                     Next To Me   
306     0wpg1fj9             Hopelessly Falling   

                                           artist_list  \
0                      [blasterjaxx, hardwell, maddix]   
1                               [ekko & sidetrack, mk]   
2    [cimone, deadline, duskee, emily makis, fricti...   
3                                       [lexed, oshan]   
4                            [bad computer, ryan coss]   
..                                                 ...   
302                                      [retrovision]   
303                                      [exyl, nasko]   
304                                  [matisse & sadko]   
305                      [don diablo, gaby gerlis, mk]   
306                   [enrem, jonah sky, rico 56, ubo]   

                              genre                 label  \
0                     [hard techno]            [revealed]   
1                     [drum & bass]         [[no labels]]   
2                     [drum & bass]        [shogun audio]   
3    [future house / future bounce]  [future house cloud]   
4                   [electro house]          [monstercat]   
..                              ...                   ...   
302                    [bass house]               [stmpd]   
303        [drumstep / moombahcore]         [[no labels]]   
304             [progressive house]               [stmpd]   
305  [future house / future bounce]             [hexagon]   
306                 [melodic house]     [monstercat silk]   

                           title  tiktok_views_total  \
0                             16            0.195087   
1    17 ekko & sidetrack bootleg            0.000000   
2             never let you down            0.003769   
3                           4 am            0.000000   
4                             4d            0.002297   
..                           ...                 ...   
302              can you feel it            0.000000   
303            curse of the moai            0.000000   
304                     himalaya            0.005572   
305                   next to me            0.001739   
306           hopelessly falling            0.000000   

     youtube_short_views_total       PWR  
0                     0.008558  0.105009  
1                     0.000000  0.000000  
2                     0.013441  0.008440  
3                     0.000000  0.000000  
4                     0.034079  0.017645  
..                         ...       ...  
302                   0.000000  0.000000  
303                   0.000000  0.000000  
304                   0.000130  0.002944  
305                   0.000000  0.000899  
306                   0.000000  0.000000  

[307 rows x 9 columns]

#### Graphical Representation

In [330]:
bar_pr_sho = plot_bar(dataframe=pr_sho.sort_values('PWR', ascending=False).head(20),
                      x='songstats_title',
                      y='PWR',
                      title='Shorts Power Ranking (Top 20)',
                      x_label='Track',
                      y_label='Power',
                      col_range=(pr_sho.PWR.min(), pr_sho.PWR.max()))
bar_pr_sho.show()

### Streams
#### Rescale statistics

In [319]:
data_str = rescale_stats(data_str, general_field)
data_str

songstats_id                songstats_title  \
0       qmr6e0bx                             16   
1       cxsruom1  17 (Ekko & Sidetrack Bootleg)   
2       847p1x5v             Never Let You Down   
3       4owlyi76                           4 AM   
4       a3ok10ce                             4D   
..           ...                            ...   
302     cqdneir6                Can You Feel It   
303     nwqit015              Curse of the Moai   
304     em4stqoh                       Himalaya   
305     wx93mcy6                     Next To Me   
306     0wpg1fj9             Hopelessly Falling   

                                           artist_list  \
0                      [blasterjaxx, hardwell, maddix]   
1                               [ekko & sidetrack, mk]   
2    [cimone, deadline, duskee, emily makis, fricti...   
3                                       [lexed, oshan]   
4                            [bad computer, ryan coss]   
..                                                 ...   
302                                      [retrovision]   
303                                      [exyl, nasko]   
304                                  [matisse & sadko]   
305                      [don diablo, gaby gerlis, mk]   
306                   [enrem, jonah sky, rico 56, ubo]   

                              genre                 label  \
0                     [hard techno]            [revealed]   
1                     [drum & bass]         [[no labels]]   
2                     [drum & bass]        [shogun audio]   
3    [future house / future bounce]  [future house cloud]   
4                   [electro house]          [monstercat]   
..                              ...                   ...   
302                    [bass house]               [stmpd]   
303        [drumstep / moombahcore]         [[no labels]]   
304             [progressive house]               [stmpd]   
305  [future house / future bounce]             [hexagon]   
306                 [melodic house]     [monstercat silk]   

                           title  soundcloud_streams_total  \
0                             16                  3.780760   
1    17 ekko & sidetrack bootleg                  7.530528   
2             never let you down                  0.107147   
3                           4 am                  0.060915   
4                             4d                  1.659776   
..                           ...                       ...   
302              can you feel it                  0.000000   
303            curse of the moai                  0.000000   
304                     himalaya                  0.123286   
305                   next to me                  0.000000   
306           hopelessly falling                  0.105649   

     spotify_streams_total  youtube_video_views_total  
0                 6.027314                   7.397974  
1                 0.000000                   3.533914  
2                 0.071527                   0.670884  
3                 0.009337                   0.092337  
4                 0.916122                   2.203943  
..                     ...                        ...  
302               0.036469                   0.195925  
303               0.128313                   0.000000  
304               0.098383                   0.775175  
305               0.654118                   0.170699  
306               0.009638                   0.035536  

[307 rows x 9 columns]

#### Weighting by recorded by stats

In [334]:
pr_str, w_str = compute_power(data_str, general_field)  # 'w_str' is calculated for the final step of the Power Ranking
pr_str

songstats_id                songstats_title  \
0       qmr6e0bx                             16   
1       cxsruom1  17 (Ekko & Sidetrack Bootleg)   
2       847p1x5v             Never Let You Down   
3       4owlyi76                           4 AM   
4       a3ok10ce                             4D   
..           ...                            ...   
302     cqdneir6                Can You Feel It   
303     nwqit015              Curse of the Moai   
304     em4stqoh                       Himalaya   
305     wx93mcy6                     Next To Me   
306     0wpg1fj9             Hopelessly Falling   

                                           artist_list  \
0                      [blasterjaxx, hardwell, maddix]   
1                               [ekko & sidetrack, mk]   
2    [cimone, deadline, duskee, emily makis, fricti...   
3                                       [lexed, oshan]   
4                            [bad computer, ryan coss]   
..                                                 ...   
302                                      [retrovision]   
303                                      [exyl, nasko]   
304                                  [matisse & sadko]   
305                      [don diablo, gaby gerlis, mk]   
306                   [enrem, jonah sky, rico 56, ubo]   

                              genre                 label  \
0                     [hard techno]            [revealed]   
1                     [drum & bass]         [[no labels]]   
2                     [drum & bass]        [shogun audio]   
3    [future house / future bounce]  [future house cloud]   
4                   [electro house]          [monstercat]   
..                              ...                   ...   
302                    [bass house]               [stmpd]   
303        [drumstep / moombahcore]         [[no labels]]   
304             [progressive house]               [stmpd]   
305  [future house / future bounce]             [hexagon]   
306                 [melodic house]     [monstercat silk]   

                           title  soundcloud_streams_total  \
0                             16                  3.780760   
1    17 ekko & sidetrack bootleg                  7.530528   
2             never let you down                  0.107147   
3                           4 am                  0.060915   
4                             4d                  1.659776   
..                           ...                       ...   
302              can you feel it                  0.000000   
303            curse of the moai                  0.000000   
304                     himalaya                  0.123286   
305                   next to me                  0.000000   
306           hopelessly falling                  0.105649   

     spotify_streams_total  youtube_video_views_total       PWR  
0                 6.027314                   7.397974  5.790382  
1                 0.000000                   3.533914  3.608242  
2                 0.071527                   0.670884  0.290303  
3                 0.009337                   0.092337  0.054405  
4                 0.916122                   2.203943  1.597467  
..                     ...                        ...       ...  
302               0.036469                   0.195925  0.080123  
303               0.128313                   0.000000  0.043257  
304               0.098383                   0.775175  0.340573  
305               0.654118                   0.170699  0.279610  
306               0.009638                   0.035536  0.049009  

[307 rows x 10 columns]

#### Graphical Representation

In [337]:
bar_pr_str = plot_bar(dataframe=pr_str.sort_values('PWR', ascending=False).head(20),
                      x='songstats_title',
                      y='PWR',
                      title='Streams Power Ranking (Top 20)',
                      x_label='Track',
                      y_label='Power',
                      col_range=(pr_str.PWR.min(), pr_str.PWR.max()))
bar_pr_str.show()